In [65]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
from langchain_groq import  ChatGroq
api = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key = api, model = "llama3-70b-8192")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027660B994E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027660B9A170>, model_name='llama3-70b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [67]:
from langchain.schema import (
    AIMessage,SystemMessage,HumanMessage
)

In [68]:
speech =  """ I have a dream that one day down in Alabama, with its vicious racists, with its governor having his lips dripping with the words of interposition and nullification – one day right there in Alabama little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.

I have a dream today.

I have a dream that one day every valley shall be exalted, and every hill and mountain shall be made low, the rough places will be made plain, and the crooked places will be made straight, and the glory of the Lord shall be revealed and all flesh shall see it together.

This is our hope. This is the faith that I go back to the South with. With this faith we will be able to hew out of the mountain of despair a stone of hope. With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood. With this faith we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day.

This will be the day, this will be the day when all of God’s children will be able to sing with new meaning “My country ’tis of thee, sweet land of liberty, of thee I sing. Land where my father’s died, land of the Pilgrim’s pride, from every mountainside, let freedom ring!  """

In [69]:
chat = [
    SystemMessage(content="You are Expert in Text Summarization"),
    HumanMessage(content=f"Provide me a Summary Of the {speech}")
]

In [70]:
llm.get_num_tokens(speech)

316

In [71]:
llm(chat)

AIMessage(content='Here is a summary of the passage:\n\nMartin Luther King Jr. expresses his dream of a future where racial segregation and discrimination are eradicated. He envisions a day when black and white children can join hands as equals in Alabama, a state notorious for its racist governor and violent racists. King dreams of a world where valleys are exalted, hills are made low, and the glory of the Lord is revealed to all. With faith, he believes that people can work together to overcome despair, transform discord into harmony, and ultimately achieve freedom and equality. He concludes by referencing the American national anthem, envisioning a day when all people can sing with pride and freedom, and freedom rings from every mountain.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 334, 'total_tokens': 476, 'completion_time': 0.405714286, 'prompt_time': 0.019288092, 'queue_time': 0.00048420400000000197, 'total_time': 0.425002

In [72]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

temp = """ 
Write a Summary of following speech:
speech:{speech}
Tanslate the precise Summary to {language}
""" 
prompt = PromptTemplate(template=temp, input_variables=["speech", "language"])
prompt

PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template=' \nWrite a Summary of following speech:\nspeech:{speech}\nTanslate the precise Summary to {language}\n')

In [73]:
complete = prompt.format(speech = speech, language= "spanish")

In [74]:
llm.get_num_tokens(complete)

339

In [75]:
llm_chain = LLMChain(llm = llm, prompt= prompt)
summary = llm_chain.run({'speech':speech,'language':'german'})
summary

'Here is a summary of the speech:\n\nThe speaker has a dream that one day, people of all races will be able to come together in unity and equality, despite the current racism and injustices. He envisions a future where people can join hands and work together to create a more just and free society. He believes that with faith, people can overcome the challenges and struggles of the present and create a brighter future for all. He dreams of a day when all people can sing with pride and freedom, and when freedom rings out across the land.\n\nGerman translation:\n\nDer Sprecher hat einen Traum, dass eines Tages Menschen aller Rassen in Einigkeit und Gleichheit zusammenkommen können, trotz der aktuellen Rassismus und Ungerechtigkeiten. Er stellt sich eine Zukunft vor, in der Menschen Hände halten und gemeinsam arbeiten können, um eine gerechtere und freiere Gesellschaft zu schaffen. Er glaubt, dass mit Glauben Menschen die Herausforderungen und Kämpfe der Gegenwart überwinden und eine besse

## STUFF DOCUMENT TEXT SUMMARIZATION


In [76]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("speeches.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': 'speeches.pdf', 'page': 0}, page_content='Hitler’s Speeches \n \nA. Speech at Munich on March 15, 1929 to the German citizens (voters) and military \nIf men wish to live, then they are forced to kill others. The entire struggle for survival is a \nconquest of the means of existence, which in turn results in the elimination of others from\u2028these same \nsources of subsistence. As long as there are peoples on this earth, there will be nations against nations and \nthey will be forced to protect their vital rights in the same way as the individual is forced to protect his \nrights.  \nOne is either the hammer or the anvil. We confess that it is our purpose to prepare the German \npeople again for the role of the hammer. We admit freely and openly that if our movement is victorious, \nwe will be concerned day and night with the question of how to produce the armed forces, which are \nforbidden us by the peace treaty [Treaty of Versailles]. We solemnly confe

In [77]:
template = """ 
Write a concise and short summary of following speech ,
speech:{text}
"""
prompt = PromptTemplate(input_variables = ['text'],template = template)
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=' \nWrite a concise and short summary of following speech ,\nspeech:{text}\n')

In [78]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type='stuff',prompt = prompt, verbose=True)
zum = chain.run(docs)
zum



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Write a concise and short summary of following speech ,
speech:Hitler’s Speeches 
 
A. Speech at Munich on March 15, 1929 to the German citizens (voters) and military 
If men wish to live, then they are forced to kill others. The entire struggle for survival is a 
conquest of the means of existence, which in turn results in the elimination of others from these same 
sources of subsistence. As long as there are peoples on this earth, there will be nations against nations and 
they will be forced to protect their vital rights in the same way as the individual is forced to protect his 
rights.  
One is either the hammer or the anvil. We confess that it is our purpose to prepare the German 
people again for the role of the hammer. We admit freely and openly that if our movement is victorious, 
we will be concerned day and night with the question of how to produce the armed forces, wh

'Here is a concise and short summary of Hitler\'s speeches:\n\nThe summary of Hitler\'s speeches reveals his aggressive and authoritarian ideology. In his speeches, he emphasizes the need for Germany to assert its dominance and protect its vital rights, even if it means violating the Treaty of Versailles. He promotes the idea of conquest and elimination of others to ensure Germany\'s survival. Hitler also stresses the importance of obedience and loyalty to the state, portraying himself as a strong leader who will guide Germany to greatness. He uses rhetorical devices such as metaphor (e.g., "hammer or anvil") and repetition to emphasize his points. The tone of his speeches is assertive, nationalistic, and sometimes messianic, appealing to the emotions and sense of duty of his audience. Overall, Hitler\'s speeches are a call to action, urging Germans to support his vision of a powerful and unified Germany.'

In [79]:
## MAP REDUCE

from langchain.text_splitter import RecursiveCharacterTextSplitter

text = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 50).split_documents(docs)
text

[Document(metadata={'source': 'speeches.pdf', 'page': 0}, page_content='Hitler’s Speeches \n \nA. Speech at Munich on March 15, 1929 to the German citizens (voters) and military \nIf men wish to live, then they are forced to kill others. The entire struggle for survival is a \nconquest of the means of existence, which in turn results in the elimination of others from\u2028these same \nsources of subsistence. As long as there are peoples on this earth, there will be nations against nations and'),
 Document(metadata={'source': 'speeches.pdf', 'page': 0}, page_content='they will be forced to protect their vital rights in the same way as the individual is forced to protect his \nrights.  \nOne is either the hammer or the anvil. We confess that it is our purpose to prepare the German \npeople again for the role of the hammer. We admit freely and openly that if our movement is victorious, \nwe will be concerned day and night with the question of how to produce the armed forces, which are'),


In [80]:
c_prompt = """
Summarize the below speech:
Speech:{text}
Summary:
""" 
map = PromptTemplate(input_variables = ['text'], template = c_prompt)

final = """
provide the final sumaary of entire speech with important points.
Generic title,
crisp summary,
in form of numbers
speech: {text}
"""
final_prompt = PromptTemplate(input_variables = ['text'], template = final)

In [84]:
summary_chain = load_summarize_chain(
    llm = llm,
    chain_type="map_reduce",
    map_prompt = map,
    combine_prompt = final_prompt,
    verbose= True
)

output = summary_chain.run(text)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Summarize the below speech:
Speech:Hitler’s Speeches 
 
A. Speech at Munich on March 15, 1929 to the German citizens (voters) and military 
If men wish to live, then they are forced to kill others. The entire struggle for survival is a 
conquest of the means of existence, which in turn results in the elimination of others from these same 
sources of subsistence. As long as there are peoples on this earth, there will be nations against nations and
Summary:

Prompt after formatting:

Summarize the below speech:
Speech:they will be forced to protect their vital rights in the same way as the individual is forced to protect his 
rights.  
One is either the hammer or the anvil. We confess that it is our purpose to prepare the German 
people again for the role of the hammer. We admit freely and openly that if our movement is victorious, 
we will be concerned day and night with the qu

Token indices sequence length is longer than the specified maximum sequence length for this model (1235 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

provide the final sumaary of entire speech with important points.
Generic title,
crisp summary,
in form of numbers
speech: Here is a summary of the speech:

In a speech on March 15, 1929, in Munich, Hitler told German citizens and military personnel that in order to survive, people must be willing to kill others. He believed that the struggle for existence is a fight for scarce resources, which inevitably leads to the elimination of others. He concluded that as long as there are different peoples on earth, there will be conflict between nations.

The speech suggests that nations, like individuals, must defend their essential rights and interests. The speaker believes that Germany must take a proactive and strong stance, rather than being passive and weak. They acknowledge that their goal is to prepare the German people to be a dominant force ("the hammer") rather than being oppressed ("the anvil"). If their

"Here is a crisp summary of the speech in numbers:\n\n**Generic Title:** Hitler's Speeches on Nationalism and Authoritarianism\n\n**Crisp Summary:**\n\n1. **Survival through conflict**: Hitler believes that nations must be willing to kill others to survive, as the struggle for existence leads to the elimination of others.\n2. **Defend essential rights**: Nations must defend their essential rights and interests, and Germany must take a proactive and strong stance.\n3. **Rejection of Treaty of Versailles**: Hitler rejects the Treaty of Versailles and encourages others to find ways to violate it.\n4. **Strengthening military**: He prioritizes strengthening the military, increasing forces, and empowering the people.\n5. **Authoritarian state**: Hitler emphasizes the importance of a strong, centralized authority in which a single will is obeyed from top to bottom.\n6. **Annexation of Austria**: He justifies the annexation of Austria by calling for a public vote on whether the unification sh

### REFINE CHAIN FOR SUMMARIZATION


In [85]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)

output = chain.run(text)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Hitler’s Speeches 
 
A. Speech at Munich on March 15, 1929 to the German citizens (voters) and military 
If men wish to live, then they are forced to kill others. The entire struggle for survival is a 
conquest of the means of existence, which in turn results in the elimination of others from these same 
sources of subsistence. As long as there are peoples on this earth, there will be nations against nations and"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: Here is a concise summary:

In a 1929 speech in Munich, Hitler told Germans that survival requires conquest and elimination of others to secure resources, and that nations will always be in conflict with each other.
We have the oppor